In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import numpy as np
import pandas as pd
import sklearn

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
train_data.describe()


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [6]:
train_data.isnull().sum()


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
def cleantrain(data): 
    
    data.drop(["PassengerId", 'Name'], axis = 1, inplace = True)
    data = pd.get_dummies(data, columns= ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
                                                 , drop_first= True)
    data = pd.concat([data, pd.get_dummies(data['Cabin'].str[0], drop_first = True)], axis = 1)
    data = pd.concat([data, pd.get_dummies(data['Cabin'].str[-1], drop_first = True)], axis = 1)
    data.drop('Cabin', axis = 1, inplace= True)
    data['Age'].fillna(data['Age'].median(), inplace = True)
    data['RoomService'].fillna(data['RoomService'].median(), inplace = True)
    data['FoodCourt'].fillna(data['FoodCourt'].median(), inplace = True)
    data['ShoppingMall'].fillna(data['ShoppingMall'].median(), inplace = True)
    data['Spa'].fillna(data['Spa'].median(), inplace = True)
    data['VRDeck'].fillna(data['VRDeck'].median(), inplace = True)

    return data

In [8]:
def cleantest(data): 
    
    data.drop(['Name'], axis = 1, inplace = True)
    data = pd.get_dummies(data, columns= ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
                                                 , drop_first= True)
    data = pd.concat([data, pd.get_dummies(data['Cabin'].str[0], drop_first = True)], axis = 1)
    data = pd.concat([data, pd.get_dummies(data['Cabin'].str[-1], drop_first = True)], axis = 1)
    data.drop('Cabin', axis = 1, inplace= True)
    data['Age'].fillna(data['Age'].median(), inplace = True)
    data['RoomService'].fillna(data['RoomService'].median(), inplace = True)
    data['FoodCourt'].fillna(data['FoodCourt'].median(), inplace = True)
    data['ShoppingMall'].fillna(data['ShoppingMall'].median(), inplace = True)
    data['Spa'].fillna(data['Spa'].median(), inplace = True)
    data['VRDeck'].fillna(data['VRDeck'].median(), inplace = True)

    return data

In [9]:
train= cleantrain(train_data)
test=cleantest(test_data)

In [10]:
train.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,...,Destination_TRAPPIST-1e,VIP_True,B,C,D,E,F,G,T,S
0,39.0,0.0,0.0,0.0,0.0,0.0,False,1,0,0,...,1,0,1,0,0,0,0,0,0,0
1,24.0,109.0,9.0,25.0,549.0,44.0,True,0,0,0,...,1,0,0,0,0,0,1,0,0,1
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,1,0,0,...,1,1,0,0,0,0,0,0,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,16.0,303.0,70.0,151.0,565.0,2.0,True,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [11]:
test.isnull().sum()


PassengerId                  0
Age                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
HomePlanet_Europa            0
HomePlanet_Mars              0
CryoSleep_True               0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
VIP_True                     0
B                            0
C                            0
D                            0
E                            0
F                            0
G                            0
T                            0
S                            0
dtype: int64

In [12]:
X = train.drop('Transported', axis = 1)
y = train['Transported']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142)

In [13]:
X.shape,y.shape

((8693, 20), (8693,))

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
lr=LogisticRegression()
lr.fit(X_train,y_train)
prediction=lr.predict(X_test)
print(accuracy_score(y_test,prediction))

0.7860839562967222


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

rf=RandomForestClassifier(n_estimators=75,max_depth=10,random_state=15)
rf.fit(X_train,y_train)
prediction=rf.predict(X_test)
print(accuracy_score(y_test,prediction))

0.8033352501437608


In [16]:
subid = test.PassengerId
test.drop('PassengerId', inplace=True, axis=1)

rf.fit(X,y)
pred = rf.predict(test)
pred = pred.astype(bool)
output = pd.DataFrame({'PassengerId': subid,'Transported': pred.squeeze()})
output.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [17]:
output.shape

(4277, 2)

In [18]:
submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
submission['Transported'] = pred
submission.to_csv('/kaggle/working/submission.csv', index=False)
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
